In [ ]:
!pip install --user --upgrade pip

In [ ]:
!pip install kfp --upgrade 

In [ ]:
# confirm the kfp sdk
!pip show kfp

Name: kfp
Version: 1.8.12
Summary: KubeFlow Pipelines SDK
Home-page: https://github.com/kubeflow/pipelines
Author: The Kubeflow Authors
Author-email: None
License: UNKNOWN
Location: /root/.local/lib/python3.7/site-packages
Requires: requests-toolbelt, typer, PyYAML, jsonschema, Deprecated, kfp-pipeline-spec, google-cloud-storage, cloudpickle, google-api-python-client, kubernetes, kfp-server-api, google-api-core, tabulate, docstring-parser, absl-py, typing-extensions, strip-hints, google-auth, click, fire, protobuf, uritemplate, pydantic
Required-by: 


## Import kubeflow pipeline libraries

In [ ]:
import kfp
import kfp.components as comp
import kfp.dsl as dsl
from kfp.components import InputPath, OutputPath
from typing import NamedTuple

## Kubeflow pipeline component creation

Component 1: Download the digits Dataset

In [ ]:
# download data step
def download_data(download_link: str, data_path: OutputPath(str)):
    import zipfile
    import sys, subprocess;
    subprocess.run(["python", "-m", "pip", "install", "--upgrade", "pip"])
    subprocess.run([sys.executable, "-m", "pip", "install", "wget"])
    import wget
    import os

    if not os.path.exists(data_path):
        os.makedirs(data_path)

    # download files
    wget.download(download_link.format(file='train'), f'{data_path}/train_csv.zip')
    wget.download(download_link.format(file='test'), f'{data_path}/test_csv.zip')
    
    with zipfile.ZipFile(f"{data_path}/train_csv.zip","r") as zip_ref:
        zip_ref.extractall(data_path)
        
    with zipfile.ZipFile(f"{data_path}/test_csv.zip","r") as zip_ref:
        zip_ref.extractall(data_path)
    
    return(print('Done!'))

Component 2: load the digits Dataset

In [ ]:
# load data

def load_data(data_path: InputPath(str), 
              load_data_path: OutputPath(str)):
    
    # import Library
    import sys, subprocess;
    subprocess.run(["python", "-m", "pip", "install", "--upgrade", "pip"])
    subprocess.run([sys.executable, '-m', 'pip', 'install','pandas'])
    # import Library
    import os, pickle;
    import pandas as pd
    import numpy as np

    #importing the data
    # Data Path
    train_data_path = data_path + '/train.csv'
    test_data_path = data_path + '/test.csv'

    # Loading dataset into pandas 
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)
    
    # join train and test together
    ntrain = train_df.shape[0]
    ntest = test_df.shape[0]
    all_data = pd.concat((train_df, test_df)).reset_index(drop=True)
    print("all_data size is : {}".format(all_data.shape))
    
    #creating the preprocess directory
    os.makedirs(load_data_path, exist_ok = True)
    
    #Save the combined_data as a pickle file to be used by the preprocess component.
    with open(f'{load_data_path}/all_data', 'wb') as f:
        pickle.dump((ntrain, all_data), f)
    
    return(print('Done!'))

Component 3: Preprocess the digits Dataset

In [ ]:
# preprocess data

def preprocess_data(load_data_path: InputPath(str), 
                    preprocess_data_path: OutputPath(str)):
    # import Library
    import sys, subprocess;
    subprocess.run(["python", "-m", "pip", "install", "--upgrade", "pip"])
    subprocess.run([sys.executable, '-m', 'pip', 'install','pandas'])
    subprocess.run([sys.executable, '-m', 'pip', 'install','scikit-learn'])
    import os, pickle;
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import OneHotEncoder
    import h5py
    import pandas as pd
    import numpy as np

    import matplotlib.pyplot as plt
    import pickle
    

    filename = f"{load_data_path}/VDISC_train.hdf5"
    data_train = {}

    with h5py.File(filename, "r") as f:
        column_names = list(f.keys())
        for column in column_names:
            data_train[column] = list(f[column])

    df_train = pd.DataFrame(data_train)
    df_train_cwe = df_train[df_train["CWE-119"] | df_train["CWE-120"] | df_train["CWE-469"] | df_train["CWE-476"] | df_train["CWE-other"]]

    df_train_clean = df_train.drop(df_train_cwe.index)

    df_train_clean_sample = df_train_clean.sample(df_train_cwe.shape[0], random_state=43) #get clean data, code withou vulnerabilities
    df_train = pd.concat([df_train_cwe, df_train_clean_sample]) # combine 
    df_train = df_train.sample(frac=1).reset_index(drop=True) #shufle data

    filename_val = f"{load_data_path}/VDISC_validate.hdf5"
    data_val = {}

    with h5py.File(filename_val, "r") as f:
        column_names = list(f.keys())
        for column in column_names:
            data_val[column] = list(f[column])
    df_val = pd.DataFrame(data_val)
    df_val_cwe = df_val[df_val["CWE-119"] | df_val["CWE-120"] | df_val["CWE-469"] | df_val["CWE-476"] | df_val["CWE-other"]]
    df_val_clean = df_val.drop(df_val_cwe.index)
    df_val_clean_sample = df_val_clean.sample(df_val_cwe.shape[0], random_state=43) #get clean data, code withou vulnerabilities
    df_val = pd.concat([df_val_cwe, df_val_clean_sample]) # combine 
    df_val = df_val.sample(frac=1).reset_index(drop=True) #shufle data

    X_train = df_train["functionSource"]
    y_train = df_train.drop(columns="functionSource")

    X_val = df_val["functionSource"]
    y_val = df_val.drop(columns="functionSource")

    ohe = OneHotEncoder(drop="if_binary", sparse=False)
    y_train_preprocessed = ohe.fit_transform(y_train)
    y_val_preprocessed = ohe.transform(y_val)

    X_train = X_train.apply(lambda x : x.decode("utf-8"))
    X_val = X_val.apply(lambda x : x.decode("utf-8"))

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(list(X_train))

    WORDS_SIZE=10000
    tokenizer.num_words = WORDS_SIZE
    list_tokenized_train = tokenizer.texts_to_sequences(X_train)
    X_train_pad = pad_sequences(list_tokenized_train,  maxlen=INPUT_SIZE, padding='post', dtype = float)

    list_tokenized_val = tokenizer.texts_to_sequences(X_val)
    X_val_pad = pad_sequences(list_tokenized_val,  maxlen=INPUT_SIZE, padding='post', dtype = float)


    return(print('Done!'))

Component 4: ML modeling

In [ ]:
def modeling(preprocess_data_path: InputPath(str), 
            model_path: OutputPath(str)):
    
    # import Library
    import sys, subprocess;
    subprocess.run(["python", "-m", "pip", "install", "--upgrade", "pip"])
    subprocess.run([sys.executable, '-m', 'pip', 'install','pandas'])
    subprocess.run([sys.executable, '-m', 'pip', 'install','tensorflow'])
    import os, pickle;
    import numpy as np
    import tensorflow as tf
    from tensorflow import keras, optimizers
    from tensorflow.keras.metrics import SparseCategoricalAccuracy
    from tensorflow.keras.losses import SparseCategoricalCrossentropy
    from tensorflow.keras import layers

    inp_layer = layers.Input(shape=(INPUT_SIZE,))
    mid_layers = layers.Embedding(input_dim = WORDS_SIZE,
                                        output_dim = 13,
                                        weights=[random_weights],
                                        input_length = INPUT_SIZE)(inp_layer)
    mid_layers = layers.Convolution1D(filters=512, kernel_size=(9), padding='same', activation='relu')(mid_layers)
    mid_layers = layers.MaxPool1D(pool_size=5)(mid_layers)
    mid_layers = layers.Dropout(0.5)(mid_layers)
    mid_layers = layers.Flatten()(mid_layers)
    mid_layers = layers.Dense(64, activation='relu')(mid_layers)
    mid_layers = layers.Dense(16, activation='relu')(mid_layers)
    output1 = layers.Dense(2, activation='softmax')(mid_layers)
    output2 = layers.Dense(2, activation='softmax')(mid_layers)
    output3 = layers.Dense(2, activation='softmax')(mid_layers)
    output4 = layers.Dense(2, activation='softmax')(mid_layers)
    output5 = layers.Dense(2, activation='softmax')(mid_layers)
    model = Model(inp_layer,[output1,output2,output3,output4,output5])

    # Define custom optimizers
    adam = optimizers.Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999, epsilon=1, decay=0.0, amsgrad=False)

    ## Compile model with metrics
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
      
    #saving the model
    model.save(f'{model_path}/model.h5')
    class_weights = [{0: 1., 1: 5.},{0: 1., 1: 5.},{0: 1., 1: 5.},{0: 1., 1: 5.},{0: 1., 1: 5.}]

    history = model.fit(x = X_train_pad,
          y = [y_train[0], y_train[1], y_train[2], y_train[3], y_train[4]],
          validation_data = (X_val_pad, [y_val[0], y_val[1], y_val[2], y_val[3], y_val[4]]),
          epochs = 100,
          batch_size = 128,
          verbose =2,
          callbacks=[callback]
          #class_weight= dict(enumerate(class_weights))
          )    

Component 5: Prediction and evaluation

In [ ]:
def prediction(model_path: InputPath(str), 
                preprocess_data_path: InputPath(str), 
                mlpipeline_ui_metadata_path: OutputPath(str)) -> NamedTuple('conf_m_result', [('mlpipeline_ui_metadata', 'UI_metadata')]):
    
    # import Library
    import sys, subprocess;
    subprocess.run(["python", "-m", "pip", "install", "--upgrade", "pip"])
    subprocess.run([sys.executable, '-m', 'pip', 'install','scikit-learn'])
    subprocess.run([sys.executable, '-m', 'pip', 'install','pandas'])
    subprocess.run([sys.executable, '-m', 'pip', 'install','tensorflow'])
    import pickle, json;
    import pandas as  pd
    import numpy as np
    from collections import namedtuple
    from sklearn.metrics import confusion_matrix
    from tensorflow.keras.models import load_model

    #loading the X_test and y_test
    with open(f'{preprocess_data_path}/test', 'rb') as f:
        test_data = pickle.load(f)
    # Separate the X_test from y_test.
    X_test, y_test = test_data
    
    #loading the model
    model = load_model(f'{model_path}/model.h5')
    
    # prediction
    y_pred = np.argmax(model.predict(X_test), axis=-1)
    
    # confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    vocab = list(np.unique(y_test))
    
    # confusion_matrix pair dataset 
    data = []
    for target_index, target_row in enumerate(cm):
        for predicted_index, count in enumerate(target_row):
            data.append((vocab[target_index], vocab[predicted_index], count))
    
    # convert confusion_matrix pair dataset to dataframe
    df = pd.DataFrame(data,columns=['target','predicted','count'])
    
    # change 'target', 'predicted' to integer strings
    df[['target', 'predicted']] = (df[['target', 'predicted']].astype(int)).astype(str)
    
    with open(mlpipeline_ui_metadata_path, 'w') as metadata_file:
        json.dump(metadata, metadata_file)

    conf_m_result = namedtuple('conf_m_result', ['mlpipeline_ui_metadata'])
    
    return conf_m_result(json.dumps(metadata))

In [ ]:
# create light weight components
download_op = comp.create_component_from_func(download_data,base_image="python:3.7.1")
load_op = comp.create_component_from_func(load_data,base_image="python:3.7.1")
preprocess_op = comp.create_component_from_func(preprocess_data,base_image="tensorflow/tensorflow:latest")
comp.create_component_from_func(preprocess_data,)
modeling_op = comp.create_component_from_func(modeling, base_image="tensorflow/tensorflow:latest")
predict_op = comp.create_component_from_func(prediction, base_image="tensorflow/tensorflow:latest")

Create kubeflow pipeline components from images

## Kubeflow pipeline creation

In [ ]:
# define pipeline
@dsl.pipeline(name="vulnerabilities-detection-pipeline", 
              description="Performs Preprocessing, training and prediction of vulnerabilities in C/C++ code")

# Define parameters to be fed into pipeline
def vulnerabilities_recognizer_pipeline(download_link: str,
                             data_path: str,
                             load_data_path: str, 
                             preprocess_data_path: str,
                             model_path:str
                            ):


    # Create download container.
    download_container = download_op(download_link)
    # Create load container.
    load_container = load_op(download_container.output)
    # Create preprocess container.
    preprocess_container = preprocess_op(load_container.output)
    # Create modeling container.
    modeling_container = modeling_op(preprocess_container.output)
    # Create prediction container.
    predict_container = predict_op(modeling_container.output, preprocess_container.output)
    

In [ ]:
pipeline_func = vulnerabilities_recognizer_pipeline

experiment_name = 'vulnerabilities_recognizer'
run_name = pipeline_func.__name__ + ' run'

# Compile pipeline to generate compressed YAML definition of the pipeline.
kfp.compiler.Compiler().compile(pipeline_func,  '{}.zip'.format(experiment_name))